# Measure % of edits coming from users without JS [T240697](https://phabricator.wikimedia.org/T240697)

We need to find out how many people are editing with no JS support (either in a browser that doesn't have JS support, or a regular browser with JS turned off). See the parent task for more info [T234695](https://phabricator.wikimedia.org/T234695) . We just need some ballpark numbers here, sampling is fine.  

Results-  For the year 2020 :  
 snapshot % of user edits done with no JS support : **27.8 %**
 
 snapshot % of anon edits done with no JS support :  **13.48 %**   
 
 snapshot % of all edits done with no JS support :  **25.12 %**  
 

Results-  For the year 2019 :  
 snapshot % of user edits done with no JS support : **29.4 %**
 
 snapshot % of anon edits done with no JS support :  **14.54 %**   
 
 snapshot % of all edits done with no JS support :  **26.37 %**  
 
 - Note that these numbers include users that have no JS in addition to many others that have ad-blockers installed and/or  enabled DNT. We will not be able to differentiate between these users without additional instrumentation

##### We also checked the numbers by Editor Interface and got the following results: 
    
For the year 2020:       
Snapshot % of user edits done with no JS support in 2020 :   
visualeditor      5.5 %  
wikitext         30.6 %  
wikitext-2017     0.15 %   

Snapshot % of anon edits done with no JS support in 2020:     
visualeditor      1.8 %   
wikitext         15.4 %    
wikitext-2017     0.0 %

Snapshot % of all edits done with no JS support in 2020:    
visualeditor      4.47 %     
wikitext         27.86 %     
wikitext-2017     0.15 %    

For the year 2019:       
Snapshot % of user edits done with no JS support in 2019 :    
visualeditor      4.2 %   
wikitext         31.8 %      
wikitext-2017     4.62 %   

Snapshot % of anon edits done with no JS support in 2019:     
visualeditor      4.05 %    
wikitext         16.17 %   
wikitext-2017     0.0 %  

Snapshot % of all edits done with no JS support in 2019:       
visualeditor      4.17 %    
wikitext         28.76 %    
wikitext-2017     4.62 %     


The no-JS edit proportions by editing interfaces raises some interesting observations and questions :    
1. The wikitext-2017 editor is a turn-on beta preference hence the anonymous non-JS edits made with it is rightly 0%
2. We have excluded all oversampled sessions from editattemptstep to obtain these results. 
3. With the assumption that VisualEditor always requires JS to load and enable editing, could the small proportion of non-JS edits for VE that we see above be indicative of users who have blocked Client-side eventlogging ?    
4. These proportions may not apply equitably to all Wikis. 
5. The wikitext editor numbers includes users that have no-JS support in addition to many others that have ad-blockers installed and/or enabled DNT. We will not be able to differentiate between these users without additional instrumentation



In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import numpy as np
import pandas as pd
from tabulate import tabulate
from wmfdata import charting, hive, mariadb
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import df_to_remarkup, pct_str

You are using wmfdata v1.0.1, but v1.0.3 is available.

To update, run `pip install --upgrade git+https://github.com/neilpquinn/wmfdata/wmfdata.git@release`.

To see the changes, refer to https://github.com/neilpquinn/wmfdata/blob/release/CHANGELOG.md


Based on discussions with R Kaldari and D Lynch here are the recommended steps that could be used to answer each of the 3 questions in the task description:  

Snapshot % of user edits done with no JS support:  
Within a specific timespan, take:  
number of sessions where user_id !== 0, integration === page, actions include init, saveSuccess, and not ready.  
Divide by:  
number of sessions where user_id !== 0, integration === page, actions include init, ready, and saveSuccess.

Snapshot % of anon edits done with no JS support:  
Within a specific timespan, take:  
number of sessions where user_id === 0, integration === page, actions include init, saveSuccess, and not ready.  
Divide by:  
number of sessions where user_id === 0, integration === page, actions include init, ready, and saveSuccess.  

Snapshot % of all edits done with no JS support:  
Within a specific timespan, take:  
number of sessions where integration === page, actions include init, saveSuccess, and not ready.  
Divide by:  
number of sessions where integration === page, actions include init, ready, and saveSuccess.  

Note that I've decided not to worry about which editor the user is using (editor_interface), which should simplify things a bit.

In [55]:
edit_sessions_query_2020= '''

-- We are only interested in sessions with saveSuccess events i.e. saved edits -- 

WITH saveSuccess_sessions as (
SELECT distinct event.editing_session_id AS ss_session_id,
  event.user_id AS user_id
FROM event_sanitized.editattemptstep
WHERE event.integration = 'page' 
  AND year = 2020  
  AND NOT event.is_oversample -- Taking out Oversampled edits
  AND event.action = 'saveSuccess' 
), 


-- Now we will use sessions without 'Ready' events to identify user edits done with no JS support -- 

ready_sessions as (
SELECT event.editing_session_id AS ready_session_id ,
  event.action as ready_action
FROM event_sanitized.editattemptstep eas
WHERE event.integration = 'page' 
  AND year = 2020  
  AND event.action='ready'
)

-- Main Query -- 
SELECT
  SUM(CAST(user_id!=0 AND ready_action is null AS int)) AS logged_nonjs_edits,
  SUM(CAST(user_id=0 and ready_action is null AS int)) AS anon_nonjs_edits,
  SUM(CAST(user_id!=0 and ready_action='ready' AS int)) AS logged_js_edits,
  SUM(CAST(user_id=0 and ready_action='ready' AS int)) AS anon_js_edits,
  SUM(CAST(ready_action is null AS int)) AS all_nonjs_edits,
  SUM(CAST(ready_action='ready' AS int)) AS all_js_edits

FROM (
  SELECT 
    user_id AS user_id , 
    ss_session_id AS ss_session_id,
    ready_action 
  FROM saveSuccess_sessions ss  
  LEFT JOIN ready_sessions ON ss.ss_session_id = ready_sessions.ready_session_id 
  --WHERE ready_sessions.ready_session_id IS NULL  
        
  GROUP BY ss.user_id, 
    ss.ss_session_id, 
    ready_sessions.ready_action
    ) edit_sessions
'''

In [56]:
edit_sessions_2020= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        "SET hive.mapred.mode=nonstrict",
        edit_sessions_query_2020 
    ]
)



Count of JS and non-JS edits with the assumption that -    
nonJS edits : number of sessions where event actions include init, saveSuccess, and not ready.     
JS edits : number of sessions where actions include init, ready, and saveSuccess.


In [57]:
edit_sessions_2020

,logged_nonjs_edits,anon_nonjs_edits,logged_js_edits,anon_js_edits,all_nonjs_edits,all_js_edits
0,783634,87733,2034214,563037,871367,2597251


### If we're interested in knowing the proportion of non-JS edits we should be dividing the number of non-JS edits by the sum of JS and non-JS edits

In [68]:
print ('Snapshot % of user edits done with no JS support in 2020: ',
       (100* (edit_sessions_2020['logged_nonjs_edits'] / 
              ( edit_sessions_2020['logged_js_edits'] + edit_sessions_2020['logged_nonjs_edits'])))
      )

Snapshot % of user edits done with no JS support in 2020:  0    27.809662
dtype: float64


In [69]:
print ('Snapshot % of anon edits done with no JS support in 2020:',
       (100* (edit_sessions_2020['anon_nonjs_edits'] / 
              ( edit_sessions_2020['anon_js_edits'] + edit_sessions_2020['anon_nonjs_edits'])))
      )

Snapshot % of anon edits done with no JS support in 2020: 0    13.481414
dtype: float64


In [70]:
print ('Snapshot % of all edits done with no JS support in 2020:',
       (100* (edit_sessions_2020['all_nonjs_edits'] / 
              ( edit_sessions_2020['all_js_edits'] + edit_sessions_2020['all_nonjs_edits'])))
      )

Snapshot % of all edits done with no JS support in 2020: 0    25.121446
dtype: float64


**Now lets look at similar numbers from 2019**

In [61]:
edit_sessions_query_2019= '''

-- We are only interested in sessions with saveSuccess events i.e. saved edits -- 

WITH saveSuccess_sessions as (
SELECT distinct event.editing_session_id AS ss_session_id,
  event.user_id AS user_id
FROM event_sanitized.editattemptstep
WHERE event.integration = 'page' 
  AND year = 2019  
  AND NOT event.is_oversample -- Taking out Oversampled edits
  AND event.action = 'saveSuccess' 
), 


-- Now we will use sessions without 'Ready' events to identify user edits done with no JS support -- 

ready_sessions as (
SELECT event.editing_session_id AS ready_session_id ,
  event.action as ready_action
FROM event_sanitized.editattemptstep eas
WHERE event.integration = 'page' 
  AND year = 2019  
  AND event.action='ready'
)

-- Main Query -- 
SELECT
  SUM(CAST(user_id!=0 AND ready_action is null AS int)) AS logged_nonjs_edits,
  SUM(CAST(user_id=0 and ready_action is null AS int)) AS anon_nonjs_edits,
  SUM(CAST(user_id!=0 and ready_action='ready' AS int)) AS logged_js_edits,
  SUM(CAST(user_id=0 and ready_action='ready' AS int)) AS anon_js_edits,
  SUM(CAST(ready_action is null AS int)) AS all_nonjs_edits,
  SUM(CAST(ready_action='ready' AS int)) AS all_js_edits

FROM (
  SELECT 
    user_id AS user_id , 
    ss_session_id AS ss_session_id,
    ready_action 
  FROM saveSuccess_sessions ss  
  LEFT JOIN ready_sessions ON ss.ss_session_id = ready_sessions.ready_session_id 
  --WHERE ready_sessions.ready_session_id IS NULL  
        
  GROUP BY ss.user_id, 
    ss.ss_session_id, 
    ready_sessions.ready_action
    ) edit_sessions
'''

In [62]:
 edit_sessions_2019= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        "SET hive.mapred.mode=nonstrict",
        edit_sessions_query_2019
    ]
)



In [63]:
edit_sessions_2019

,logged_nonjs_edits,anon_nonjs_edits,logged_js_edits,anon_js_edits,all_nonjs_edits,all_js_edits
0,1665592,210986,3998811,1239112,1876578,5237923


In [64]:
print ('Snapshot % of user edits done with no JS support in 2019 :',
       (100* (edit_sessions_2019['logged_nonjs_edits'] / 
              ( edit_sessions_2019['logged_js_edits'] + edit_sessions_2019['logged_nonjs_edits'])))
      )

Snapshot % of user edits done with no JS support in 2019 : 0    29.404546
dtype: float64


In [65]:
print ('Snapshot % of anon edits done with no JS support in 2019 :',
       (100* (edit_sessions_2019['anon_nonjs_edits'] / 
              ( edit_sessions_2019['anon_js_edits'] + edit_sessions_2019['anon_nonjs_edits'])))
      )

Snapshot % of anon edits done with no JS support in 2019 : 0    14.549775
dtype: float64


In [66]:
print ('Snapshot % of all edits done with no JS support in 2019 :',
       (100* (edit_sessions_2019['all_nonjs_edits'] / 
              ( edit_sessions_2019['all_js_edits'] + edit_sessions_2019['all_nonjs_edits'])))
      )

Snapshot % of all edits done with no JS support in 2019 : 0    26.376804
dtype: float64


### Next lets look at non-JS edits proportion by editing interface
For the year 2020

In [74]:
edit_interface_query_2020= '''

-- We are only interested in sessions with saveSuccess events i.e. saved edits -- 

WITH saveSuccess_sessions as (
SELECT distinct event.editing_session_id AS ss_session_id,
  event.user_id AS user_id, 
  event.editor_interface AS editing_interface
FROM event_sanitized.editattemptstep
WHERE event.integration = 'page' 
  AND year = 2020  
  AND NOT event.is_oversample -- Taking out Oversampled edits
  AND event.action = 'saveSuccess' 
), 


-- Now we will use sessions without 'Ready' events to identify user edits done with no JS support -- 

ready_sessions as (
SELECT event.editing_session_id AS ready_session_id ,
  event.action as ready_action
FROM event_sanitized.editattemptstep eas
WHERE event.integration = 'page' 
  AND year = 2020  
  AND event.action='ready'
)

-- Main Query -- 
SELECT
  editing_interface AS editing_interface,  
  SUM(CAST(user_id!=0 AND ready_action is null AS int)) AS logged_nonjs_edits,
  SUM(CAST(user_id=0 and ready_action is null AS int)) AS anon_nonjs_edits,
  SUM(CAST(user_id!=0 and ready_action='ready' AS int)) AS logged_js_edits,
  SUM(CAST(user_id=0 and ready_action='ready' AS int)) AS anon_js_edits,
  SUM(CAST(ready_action is null AS int)) AS all_nonjs_edits,
  SUM(CAST(ready_action='ready' AS int)) AS all_js_edits

FROM (
  SELECT 
    user_id AS user_id , 
    ss_session_id AS ss_session_id,
    editing_interface,
    ready_action 
  FROM saveSuccess_sessions ss  
  LEFT JOIN ready_sessions ON ss.ss_session_id = ready_sessions.ready_session_id 
  --WHERE ready_sessions.ready_session_id IS NULL  
        
  GROUP BY ss.user_id, 
    ss.ss_session_id, 
    ss.editing_interface,
    ready_sessions.ready_action
    ) edit_sessions

GROUP BY editing_interface    
'''

In [75]:
edit_interface_sessions_2020= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        "SET hive.mapred.mode=nonstrict",
        edit_interface_query_2020 
    ]
)


In [76]:
edit_interface_sessions_2020

,editing_interface,logged_nonjs_edits,anon_nonjs_edits,logged_js_edits,anon_js_edits,all_nonjs_edits,all_js_edits
0,visualeditor,13000,1774,223169,92468,14774,315637
1,wikitext,771553,86077,1749194,471378,857630,2220572
2,wikitext-2017,99,0,65041,8,99,65049


In [94]:
edit_interface_sessions_2020=edit_interface_sessions_2020.set_index('editing_interface')

In [95]:
print ('Snapshot % of user edits done with no JS support in 2020 :',
       (100* (edit_interface_sessions_2020['logged_nonjs_edits'] / 
              ( edit_interface_sessions_2020['logged_js_edits'] + edit_interface_sessions_2020['logged_nonjs_edits'])))
      )

Snapshot % of user edits done with no JS support in 2020 : editing_interface
visualeditor      5.504533
wikitext         30.608109
wikitext-2017     0.151980
dtype: float64


In [96]:
print ('Snapshot % of anon edits done with no JS support in 2020:',
       (100* (edit_interface_sessions_2020['anon_nonjs_edits'] / 
              ( edit_interface_sessions_2020['anon_js_edits'] + edit_interface_sessions_2020['anon_nonjs_edits'])))
      )

Snapshot % of anon edits done with no JS support in 2020: editing_interface
visualeditor      1.882388
wikitext         15.441067
wikitext-2017     0.000000
dtype: float64


In [97]:
print ('Snapshot % of all edits done with no JS support in 2020:',
       (100* (edit_interface_sessions_2020['all_nonjs_edits'] / 
              ( edit_interface_sessions_2020['all_js_edits'] + edit_interface_sessions_2020['all_nonjs_edits'])))
      )

Snapshot % of all edits done with no JS support in 2020: editing_interface
visualeditor      4.471401
wikitext         27.861394
wikitext-2017     0.151962
dtype: float64


**Now let's look at the numbers for the year 2019**

In [80]:
edit_interface_query_2019= '''

-- We are only interested in sessions with saveSuccess events i.e. saved edits -- 

WITH saveSuccess_sessions as (
SELECT distinct event.editing_session_id AS ss_session_id,
  event.user_id AS user_id, 
  event.editor_interface AS editing_interface
FROM event_sanitized.editattemptstep
WHERE event.integration = 'page' 
  AND year = 2019  
  AND NOT event.is_oversample -- Taking out Oversampled edits
  AND event.action = 'saveSuccess' 
), 


-- Now we will use sessions without 'Ready' events to identify user edits done with no JS support -- 

ready_sessions as (
SELECT event.editing_session_id AS ready_session_id ,
  event.action as ready_action
FROM event_sanitized.editattemptstep eas
WHERE event.integration = 'page' 
  AND year = 2019  
  AND event.action='ready'
)

-- Main Query -- 
SELECT
  editing_interface AS editing_interface,  
  SUM(CAST(user_id!=0 AND ready_action is null AS int)) AS logged_nonjs_edits,
  SUM(CAST(user_id=0 and ready_action is null AS int)) AS anon_nonjs_edits,
  SUM(CAST(user_id!=0 and ready_action='ready' AS int)) AS logged_js_edits,
  SUM(CAST(user_id=0 and ready_action='ready' AS int)) AS anon_js_edits,
  SUM(CAST(ready_action is null AS int)) AS all_nonjs_edits,
  SUM(CAST(ready_action='ready' AS int)) AS all_js_edits

FROM (
  SELECT 
    user_id AS user_id , 
    ss_session_id AS ss_session_id,
    editing_interface,
    ready_action 
  FROM saveSuccess_sessions ss  
  LEFT JOIN ready_sessions ON ss.ss_session_id = ready_sessions.ready_session_id 
  --WHERE ready_sessions.ready_session_id IS NULL  
        
  GROUP BY ss.user_id, 
    ss.ss_session_id, 
    ss.editing_interface,
    ready_sessions.ready_action
    ) edit_sessions

GROUP BY editing_interface    
'''

In [81]:
edit_interface_sessions_2019= hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        "SET hive.mapred.mode=nonstrict",
        edit_interface_query_2019 
    ]
)


In [89]:
edit_interface_sessions_2019

,editing_interface,logged_nonjs_edits,anon_nonjs_edits,logged_js_edits,anon_js_edits,all_nonjs_edits,all_js_edits
0,visualeditor,16445,7898,372427,186791,24343,559218
1,wikitext,1644055,203088,3521362,1052300,1847143,4573662
2,wikitext-2017,5092,0,105026,21,5092,105047


In [90]:
edit_interface_sessions_2019=edit_interface_sessions_2019.set_index('editing_interface')

In [91]:
print ('Snapshot % of user edits done with no JS support in 2019 :',
       (100* (edit_interface_sessions_2019['logged_nonjs_edits'] / 
              ( edit_interface_sessions_2019['logged_js_edits'] + edit_interface_sessions_2019['logged_nonjs_edits'])))
      )

Snapshot % of user edits done with no JS support in 2019 : editing_interface
visualeditor      4.228898
wikitext         31.828118
wikitext-2017     4.624130
dtype: float64


In [98]:
print ('Snapshot % of anon edits done with no JS support in 2019:',
       (100* (edit_interface_sessions_2019['anon_nonjs_edits'] / 
              ( edit_interface_sessions_2019['anon_js_edits'] + edit_interface_sessions_2019['anon_nonjs_edits'])))
      )

Snapshot % of anon edits done with no JS support in 2019: editing_interface
visualeditor      4.056726
wikitext         16.177309
wikitext-2017     0.000000
dtype: float64


In [99]:
print ('Snapshot % of all edits done with no JS support in 2019:',
       (100* (edit_interface_sessions_2019['all_nonjs_edits'] / 
              ( edit_interface_sessions_2019['all_js_edits'] + edit_interface_sessions_2019['all_nonjs_edits'])))
      )

Snapshot % of all edits done with no JS support in 2019: editing_interface
visualeditor      4.171458
wikitext         28.768091
wikitext-2017     4.623249
dtype: float64
